In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [3]:
from langchain_community.tools import TavilySearchResults

query = "스테이크와 어울리는 와인을 추천해주세요"
web_search = TavilySearchResults(max_results=2)
search_results = web_search.invoke(query)

for result in search_results:
    print(result)
    print("-" * 100)

{'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\n\u200b\n\n이에 맞는 레드와인으로는\n\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\n\u200b\n\n![Image 7](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTEw/MDAxNjgzNTQ1MDQ3NDI0.v4ZHSvlETicMdag0DaoWGFsrTMFezoiZD_41J0Htwhsg.mPiLhr9soLZMXEOb_GJatpnqVUUNFZkrcmQRn9omaewg.JPEG.wineislikeacat/clark-douglas-7WJV-SDSj6c-unsplash.jpg?type=w80_blur)\n\n안심 스테이크와 어울리는 와인\n\n> **안심 스테이크와 어울리는 와인 품종은? 산지오베제!**\n\n고기 본연의 맛을 즐기기 가장 좋은 부위로 꼽히는 안심은 [...] ![Image 6](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTQ4/MDAxNjgzNTQ0MjY4OTc2.FasX2MpcBaCDnf0qjWA_PcbLTIlySdPLi7FFnJlTboog.AQwrp1etohVQ3fiIHA0ga9lD45uGexCOZzB_1C12eGQg.JPEG.wineislikeacat/paul-einerhand-dwtt3noJG2k-unsplash.jpg?type=w800)\n\n등심 & 채끝 스테이크와 어울리는 와인\n\n> **등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!**\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\n\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\n\n채끝은 등뼈에서 허리

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[web_search])

In [5]:
query = "안녕하세요"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls) # 사용할 tools 정보를 보여주는데 여기서는 query가 단순해서 빈리스트가 나옴
print("-" * 100)


AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--ca2b33b3-21d2-471b-b8b7-75495f567b6a-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92})
----------------------------------------------------------------------------------------------------
'안녕하세요! 어떻게 도와드릴까요?'
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


In [6]:
query = "스테이크와 어울리는 와인을 추천해주세요"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls) 
print("-" * 100)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Hhz07vJOi2yu9arcxhG5oH5Z', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 90, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0123ddd8-96e4-401f-8ffa-75d6e0352164-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_Hhz07vJOi2yu9arcxhG5oH5Z', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 27, 'total_tokens': 117})
-------------------------------------------------

In [7]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_Hhz07vJOi2yu9arcxhG5oH5Z',
 'type': 'tool_call'}

In [8]:
tool_output = web_search.invoke(tool_call["args"])
print(f"{tool_call['name']} 호출결과:")
print("-" * 100)
print(tool_output)

tavily_search_results_json 호출결과:
----------------------------------------------------------------------------------------------------
[{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n\n**추천 이유:**\n\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n**어울리는 스테이크:**\n\n*   리브아이 스테이크\n*   뉴욕 스트립\n*   티본 스테이크\n\n**추천 와인:**\n\n*   나파 밸리 카베르네 소비뇽\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img

In [9]:
from langchain_core.messages import ToolMessage
tool_message = ToolMessage(
    content=tool_output,
    tool_call_id=tool_call["id"],
    name=tool_call["name"]
)

print(tool_message)

content=[{'url': 'https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80', 'content': '1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\n------------------------------------------------\n\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\n\n**추천 이유:**\n\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\n\n**어울리는 스테이크:**\n\n*   리브아이 스테이크\n*   뉴욕 스트립\n*   티본 스테이크\n\n**추천 와인:**\n\n*   나파 밸리 카베르네 소비뇽\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.webp)![Image 12](https://blog.kaka

In [10]:
tool_message = web_search.invoke(tool_call)
print(tool_message)

content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.webp)![Image 12

In [11]:
pprint(tool_message.content)

('[{"url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 '
 '대표주자\\n------------------------------------------------\\n\\n![Image '
 '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
 '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 '
 '이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, '
 '고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   '
 '티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château '
 'Margaux) [...] ![Image '
 '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
 '11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKpr

In [12]:
tool_message = web_search.batch(ai_msg.tool_calls)
pprint(tool_message)
print("-" * 100)
pprint(tool_message[0].content)

[ToolMessage(content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.we

In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant, Today's date is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=[web_search])

llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-" * 100)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-" * 100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 가격은 얼마인가요?")

pprint(response.content)


ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_CrHLg7vMTYJ5oFhoOqKqSGEV', 'function': {'arguments': '{"query":"모엣샹동 샴페인 가격 2025년 6월"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 114, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--4b3e014f-374e-4dc7-9513-214b5a7da43f-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '모엣샹동 샴페인 가격 2025년 6월'}, 'id': 'call_CrHLg7vMTYJ5oFhoOqKqSGEV', 'type': 'tool_call'}] usage_metadata={'input_tokens': 114, 'output_tokens': 35, 'total_tokens': 149}
---------------------------------------------

In [14]:
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

@tool
def search_web(query: str) -> List[str]:
    """ tool 용도 기재 """
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    formatted_docs = "\n---\n".join([
        f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
        for doc in docs
    ])

    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."

In [15]:
print# 도구 속성
print("자료형: ")
print(type(search_web))
print("-"*100)

print("name: ")
print(search_web.name)
print("-"*100)

print("description: ")
pprint(search_web.description)
print("-"*100)

print("schema: ")
pprint(search_web.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_web
----------------------------------------------------------------------------------------------------
description: 
'tool 용도 기재'
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'tool 용도 기재 ',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_web',
 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [16]:
query = "스테이크와 어울리는 와인을 추천해주세요."
search_results = search_web.invoke(query)

print(search_results)

<Document href="https://mashija.com/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%B5%9C%EA%B3%A0%EC%9D%98-%EC%99%80%EC%9D%B8-%EB%AC%B4%EC%97%87%EC%9D%84-%EA%B3%A0%EB%A5%BC-%EA%B2%83%EC%9D%B8/"/>
카베르네 소비뇽(Cabernet Sauvignon) 및 말벡(Malbec)과 같은 전형적인 선택부터 더 가벼운 레드 와인, 심지어 화이트 와인과 맛있는 스테이크를 페어링하는 방법까지, 우리의 아카이브에서 가져온 최고의 조언과 최근 디캔터 전문가가 추천한 와인을 소개한다.

[![Image 4](https://mashija.com/wp-content/uploads/2022/01/%EC%A0%9C%EB%AA%A9%EC%9D%84-%EC%9E%85%EB%A0%A5%ED%95%B4%EC%A3%BC%EC%84%B8%EC%9A%94_-001-7.png)](https://mashija.com/wp-content/uploads/2022/01/%EC%A0%9C%EB%AA%A9%EC%9D%84-%EC%9E%85%EB%A0%A5%ED%95%B4%EC%A3%BC%EC%84%B8%EC%9A%94_-001-7.png)

**<스테이크를 곁들인 레드 와인을 위한 5가지 전형적인 선택\>** [...] • 카베르네 소비뇽(Cabernet Sauvignon)  
• 말벡(Malbec)  
• 그르나슈/쉬라즈 블렌드(Grenache / Shiraz blends)  
• 시라/쉬라즈(Syrah / Shiraz)  
• 산지오베제(Sangiovese)

육즙이 풍부한 스테이크와 맛있는 와인이 있는 저녁 식사는 적어도 고기 애호가들에게 인생의 큰 즐거움일 것이다.

와인과 음식 페어링에서 새로운 시도를 하는 것은 항상 재미있지만, 특별한 스테이크 저녁 식사를 준비할 때 고려해야 할 

In [17]:
llm_with_tools = llm.bind_tools(tools=[search_web])
query = "스테이크와 어울리는 와인을 추천해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SaR9LfH5kdGgWwyOQednj3m2', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 55, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d2d31dd9-493f-4f54-af4c-14c68f5a4d45-0', tool_calls=[{'name': 'search_web', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_SaR9LfH5kdGgWwyOQednj3m2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 23, 'total_tokens': 78})
-----------------------------------------------------------------------------------

In [18]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field
from typing import List

def search_wiki(input_data: dict) -> List[Document]:
    query = input_data["query"]
    k = input_data.get("k", 2)
    wiki_loader = WikipediaLoader(query=query, load_max_docs=k, lang="ko")
    wiki_docs = wiki_loader.load()
    return wiki_docs

class WikiSearchSchema(BaseModel):
    query: str = Field(..., description="The query to search for in Wikipedia")
    k: int = Field(2, description="The number of documentts to return")

runnable = RunnableLambda(search_wiki)
wiki_search = runnable.as_tool(
    name="wiki_search",
    description=dedent("""
    Hello
"""),  
    args_schema=WikiSearchSchema
)

C:\Users\12wkd\AppData\Local\Temp\ipykernel_35732\1538975992.py:19: LangChainBetaWarning: This API is in beta and may change in the future.
  wiki_search = runnable.as_tool(


In [19]:
query = "파스타의 유래"
wiki_results = wiki_search.invoke({"query":query})

for result in wiki_results:
    print(result)
    print("-"*100)

page_content='이탈리아 요리(Italia 料理, 이탈리아어: cucina italiana 쿠치나 이탈리아나[*])는 남유럽에 있는 이탈리아의 요리이다. 기원전 4세기부터 다양한 사회·정치 변화와 함께 발전해 왔다. 특히 유럽인들이 세계 각지로 진출한 이후 신대륙에서 감자, 토마토, 단고추, 옥수수 등의 다양한 식재료가 유입되면서 이탈리아 요리는 큰 변화를 겪었다. 이런 재료는 18세기 이전까지는 보급되지 않았다.
이탈리아 요리는 각 지역마다 고유한 특색이 있고 다양하지만 크게 북부와 남부로 구분한다. 다른 나라와 국경을 맞대던 북부는 산업화하여 호황을 누리면서 농업이 발달한 덕분에 쌀이나 유제품이 사용되는 요리가 많지만 경제적으로 침체됐던 남부는 지역 특산물인 올리브와 토마토, 모짜렐라 치즈나 해산물을 활용한 요리가 많다.
간결함으로 귀결되는 특징이 있어 많은 경우 4개~8개 정도의 재료로 조리되며 정교한 준비 과정보다는 재료의 질에 따라 맛이 좌우된다. 치즈와 포도주는 이탈리아 요리의 핵심을 차지하며 여러 종류가 존재하는데 식재료와 치즈의 차이는 파스타의 종류와 요리에 기본이 되는 수프, 소스의 차이를 뜻한다.
2013년 CNN은 세계 최고의 요리로 이탈리아 요리를 선정했다.


== 역사 ==
대략 기원전 4세기부터 시작되었다고 여겨진다. 여러 세기 동안 주변 지역과 정복자, 유명한 요리사, 정치 변동 및 신대륙의 발견 등 매우 다양한 요소가 이탈리아 요리의 형성에 영향을 끼쳤다.


=== 고대 ===
이탈리아 요리에 대해 최초의 저술을 남긴 사람은 그리스계 시칠리아 사람인 아케스트라투스이다. 기원전 4세기에 지금의 시라쿠사 지역에 살던 그는 “계절에 맞는 최상의 재료”를 사용하는 것에 대한 시를 남겼다. 그는 또한 향신료나 허브, 양념 따위를 사용한 음식과 그 중요성, 생선 요리 등에 대해 서술했다. 특정 양념이나 향신료가 지나쳐서는 안 된다는 것 또한 집필 내용에 포함됐지만 470가지의 요리법을 담은 책인 《데 레 코퀴나리아》(De re c

In [20]:
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_search])
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요. "
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BOGYtCHb940MEXgElVzDt7ch', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_mhAiakLL9k21VY7q1S5o4s00', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 112, 'total_tokens': 164, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8192697e-61b1-405b-85df-19a985ff570c-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_BOGYtCHb940MEXgElVzDt7ch', 'type': 'tool_call'}, {'name': 'wiki_search', 'args': {'qu

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import WikipediaLoader
from operator import itemgetter

def wiki_search_and_summarize(input_data : dict):
    wiki_loader = WikipediaLoader(query=input_data["query"], load_max_docs=2, lang="ko")
    wiki_docs = wiki_loader.load()

    formatted_docs = [
        f'<Documnet source="{doc.metadata["source"]}"/>\n{doc.page_content}\n</Document>'
        for doc in wiki_docs
    ]

    return formatted_docs

summary_prompt = ChatPromptTemplate.from_template(
    "Summarize the following text in a concise manner:\n\n{context}\n\nSummary:"
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summary_chain = (
    {"context":RunnableLambda(wiki_search_and_summarize)}
    | summary_prompt | llm | StrOutputParser()
)

summarize_text = summary_chain.invoke({"query":"파스타의 유래"})
pprint(summarize_text)

('이탈리아 요리(쿠치나 이탈리아나)는 기원전 4세기부터 발전해온 남유럽의 요리로, 지역마다 고유한 특색이 있으며 북부와 남부로 구분된다. '
 '북부는 쌀과 유제품을, 남부는 올리브와 토마토, 해산물을 주로 사용한다. 이탈리아 요리는 간결함을 특징으로 하며, 재료의 질이 맛에 큰 '
 '영향을 미친다. 역사적으로 그리스와 로마의 영향을 받았고, 중세에는 아랍 문화가 유입되면서 변화가 있었다. 2013년 CNN은 이탈리아 '
 '요리를 세계 최고의 요리로 선정했다. 오르조(리소니)는 이탈리아의 파스타로, 보리에서 유래했으며 수프와 함께 먹는다.')


In [22]:
class WikiSummarySchema(BaseModel):
    query: str = Field(...,description="The query to search for in wikipedia")

wiki_summary = summary_chain.as_tool( 
    name="wiki_summary",
    description=dedent("""
        Use this tool when you need to search for information on Wikipedia.
        It searches for Wikipedia articles related to the user's query and returns
        a summarized text. This tool is useful when general knowledge
        or background information is required.
    """),
    args_schema=WikiSummarySchema
)

print("자료형: ")
print(type(wiki_summary))
print("-" * 100)

print("name: ")
print(wiki_summary.name)
print("-"*100)

print("description: ")
pprint(wiki_summary.description)
print("-"*100)

print("schema: ")
pprint(wiki_summary.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
wiki_summary
----------------------------------------------------------------------------------------------------
description: 
('Use this tool when you need to search for information on Wikipedia.\n'
 "It searches for Wikipedia articles related to the user's query and returns\n"
 'a summarized text. This tool is useful when general knowledge\n'
 'or background information is required.')
----------------------------------------------------------------------------------------------------
schema: 
{'properties': {'query': {'description': 'The query to search for in wikipedia',
                          'title': 'Query',
                          'type': 'string'}},
 'required': ['query'],
 'title': 'WikiSummarySchema',
 'type': 'object'}
-----------------------------------------------------------------------------------

In [23]:
llm_with_tools = llm.bind_tools(tools=[search_web, wiki_summary])
query = "서울 강남의 유명한 파스타 맛집은 어디인가요? 그리고 파스타의 유래를 알려주세요"
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-" * 100)

pprint(ai_msg.content)
print("-" * 100)

pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W9H0dmLb9HTDlENrICZBIq1Y', 'function': {'arguments': '{"query": "서울 강남 파스타 맛집"}', 'name': 'search_web'}, 'type': 'function'}, {'id': 'call_RzZiMHQsfAazstGJmlwUw0rM', 'function': {'arguments': '{"query": "파스타"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 136, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--820f8f8e-5f64-4862-8e98-a2c437cc40d9-0', tool_calls=[{'name': 'search_web', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_W9H0dmLb9HTDlENrICZBIq1Y', 'type': 'tool_call'}, {'name': 'wiki_summary', 'args': {'

In [24]:
ai_msg.tool_calls[1]

{'name': 'wiki_summary',
 'args': {'query': '파스타'},
 'id': 'call_RzZiMHQsfAazstGJmlwUw0rM',
 'type': 'tool_call'}

In [25]:
tool_message = wiki_summary.invoke(ai_msg.tool_calls[1])

print(tool_message)
print("-" * 100)
pprint(tool_message.content)

content='파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 음식으로 여겨지며, 역사적으로 그리스와 아랍 문화의 영향을 받았다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 존재한다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있다. 투움바 파스타는 미국의 요리로, 오스트레일리아의 도시에서 유래되었으며, 매콤한 크림 소스를 사용한 두꺼운 파스타 요리로 한국에서 인기를 끌고 있다.' name='wiki_summary' tool_call_id='call_RzZiMHQsfAazstGJmlwUw0rM'
----------------------------------------------------------------------------------------------------
('파스타는 이탈리아의 주요 밀 식품으로, 듀럼밀 세몰라와 물 또는 밀가루와 달걀로 만들어지며, 삶거나 구워서 먹는다. 이탈리아의 국민 '
 '음식으로 여겨지며, 역사적으로 그리스와 아랍 문화의 영향을 받았다. 파스타는 건파스타와 생파스타로 나뉘며, 다양한 형태와 종류가 '
 '존재한다. 요리 방법으로는 삶거나 오븐에 구워내는 방식이 있다. 투움바 파스타는 미국의 요리로, 오스트레일리아의 도시에서 유래되었으며, '
 '매콤한 크림 소스를 사용한 두꺼운 파스타 요리로 한국에서 인기를 끌고 있다.')


In [26]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm_with_tools = llm.bind_tools([wiki_summary])
llm_chain = prompt | llm_with_tools

@chain
def wiki_summary_chain(user_input:str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    print("ai_msg: \n", ai_msg)
    print("-" * 100)
    tool_msgs = wiki_summary.batch(ai_msg.tool_calls, config=config)
    print("tool_msgs: \n", tool_msgs)
    print("-" * 100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = wiki_summary_chain.invoke("파스타의 유래에 대해서 알려주세요.")
pprint(response.content)

ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_sqdvuFpeHOTShrV3cKjSvLjP', 'function': {'arguments': '{"query":"파스타의 유래"}', 'name': 'wiki_summary'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 119, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--e55f7e41-2c38-405d-a20a-6b2e0254ee89-0' tool_calls=[{'name': 'wiki_summary', 'args': {'query': '파스타의 유래'}, 'id': 'call_sqdvuFpeHOTShrV3cKjSvLjP', 'type': 'tool_call'}] usage_metadata={'input_tokens': 119, 'output_tokens': 19, 'total_tokens': 138}
---------------------------------------------------------------------------------------------------

In [27]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()
print(len(documents))

1


In [28]:
from langchain_core.documents import Document

def split_menu_items(document):
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)

    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()

        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    return menu_documents

menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")

총 10개의 메뉴 항목이 처리되었습니다.


In [29]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3")

menu_db = Chroma.from_documents(
    documents=menu_documents,
    embedding=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

menu_retriever = menu_db.as_retriever(
    search_kwargs={'k':2},
)

query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색결과 : {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색결과 : 2개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크



In [30]:
menu_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

@tool
def search_menu(query: str) -> List[Document]:
    """ 
    tool 데코 용도 
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs)>0:
        return docs
    
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

print("자료형: ")
print(type(search_menu))
print("-" * 100)

print("name: ")
print(search_menu.name)
print("-" * 100)

print("description: ")
print(search_menu.description)
print("-" * 100)

print("schema: ")
print(search_menu.args_schema.schema())
print("-" * 100)


자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_menu
----------------------------------------------------------------------------------------------------
description: 
tool 데코 용도
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'tool 데코 용도 ', 'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'search_menu', 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [31]:
wine_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="wine_menu",
    persist_directory="./chroma_db"
)

@tool
def search_wine(query:str) -> List[Document]:
    """
    사용용도
    """
    docs = wine_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs
    return [Document(page_content="관련 와인 정보를 찾을 수 없습니다습니다")]

print("자료형: ")
print(type(search_wine))
print("-" * 100)

print("name: ")
print(search_wine.name)
print("-" * 100)

print("description: ")
print(search_wine.description)
print("-" * 100)

print("schema: ")
print(search_wine.args_schema.schema())
print("-" * 100)


자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_wine
----------------------------------------------------------------------------------------------------
description: 
사용용도
----------------------------------------------------------------------------------------------------
schema: 
{'description': '사용용도', 'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'search_wine', 'type': 'object'}
----------------------------------------------------------------------------------------------------


In [32]:
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print("-"*100)

pprint(ai_msg.content)
print("-"*100)

pprint(ai_msg.tool_calls)
print("-"*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zvwBeUWYHK96NpKPZ9zqdpva', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_ABaTPkA4C8Dz5kQMVjXAL24s', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 93, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7e5335a7-c78d-4079-8aa1-94c631c7c883-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_zvwBeUWYHK96NpKPZ9zqdpva', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query'

In [33]:
tools = [search_web, wiki_summary, search_wine, search_menu]
for tool in tools:
    print(tool.name)

search_web
wiki_summary
search_wine
search_menu


In [34]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.today().strftime("%Y-%m-%d")

prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=tools)
llm_chain = prompt | llm_with_tools

@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-" * 100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        
        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-" * 100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

response = restaurant_menu_chain.invoke("시그니쳐 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인을 추천해주세요")
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '시그니쳐 스테이크 가격과 특징'}, 'id': 'call_6ubxGhxCzEOWXMrwTXT5cczu', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_gpX1H8eB24aPuYTcuH9kNjaE', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격:

In [35]:
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요")
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '파스타'}, 'id': 'call_IqNaQcIjlBgDsloZbUUy82MB', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
wiki_summary: 
{'name': 'wiki_summary', 'args': {'query': '파스타'}, 'id': 'call_ASQxvFXSuuKAeIOVB0MwrC4k', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n   • 설명: 알 덴테로 삶은 링귀네 파스타에 신선한 해산물을 듬뿍 올린 메뉴입니다. 토마토 소스의 산미와 해산물의 감칠맛이 조화를 이루며, 마늘과 올리브 오일로 풍미를 더했습니다. 파슬리를 뿌려 향긋한 맛을 더합니다.'), Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n 

In [36]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=tools)
fewshot_search_chain = few_shot_prompt | llm_with_tools
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)


for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_menu', 'args': {'query': '스테이크'}, 'id': 'call_fV0arKniukszSKxlEYPY15yx', 'type': 'tool_call'}
{'name': 'search_wine', 'args': {'query': '스테이크'}, 'id': 'call_qlfhY2Rd21UOORGUBK19oegH', 'type': 'tool_call'}


In [37]:
from datetime import date
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

today = datetime.today().strftime("%Y-%m-%d")
system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system + f"Today's date is {today}."),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools=tools)
fewshot_search_chain = few_shot_prompt | llm_with_tools

@chain
def restaurant_menu_chain(user_input: str, config=RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-" * 100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "wiki_summary":
            tool_message = wiki_summary.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        
        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)
        
        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

    print("tool_msgs: \n", tool_msgs)
    print("-" * 100)
    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = restaurant_menu_chain.invoke(query)

pprint(response)

search_menu: 
{'name': 'search_menu', 'args': {'query': '스테이크'}, 'id': 'call_nGbZbt70GgXXrKPagjz3ymTN', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크 와인 추천'}, 'id': 'call_cNP3tRfZcUg0GqFLQd49ve37', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   •

In [43]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate([
    ("system", dedent("""
    You are an AI assistant providing restaurant menu information and general food-related knowledge. 
    Your main goal is to provide accurate information and effective recommendations to users.

    Key guidelines:
    1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
    2. For general food information, history, and cultural background, utilize the wiki_summary tool.
    3. For wine recommendations or food and wine pairing information, use the search_wine tool.
    4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
    5. Provide clear and concise responses based on the search results.
    6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
    7. Always maintain a helpful and professional tone.
    8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
    9. When making recommendations, briefly explain the reasons.
    10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


    Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
    Combine the tools to provide the most comprehensive and accurate answers to user queries. 
    Always strive to provide the most current and accurate information.
    """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [44]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, wiki_summary, search_wine, search_menu]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)


In [46]:
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
agent_respone = agent_executor.invoke({"input":query})



> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.')]
Invoking: `search_wine` with `{'query': '스테이크 와인 페어링'}`


[Document(metadata={}, page_content='관련 와인 정보를 찾을 수 없습니다습니다')]### 시그니처 스테이크 정보
- **가격**: 35,000원
- **주요 재료**: 소고기, 로즈마리, 그릴드 아스파라거스
- **특징**: 
  - 스테이

In [47]:
pprint(agent_respone)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.',
 'output': '### 시그니처 스테이크 정보\n'
           '- **가격**: 35,000원\n'
           '- **주요 재료**: 소고기, 로즈마리, 그릴드 아스파라거스\n'
           '- **특징**: \n'
           '  - 스테이크는 21일간 숙성된 소고기를 사용하며, 미디엄 레어로 조리됩니다.\n'
           '  - 부드러운 식감이 특징이며, 향긋한 허브와 함께 제공되어 풍미가 더욱 강조됩니다.\n'
           '  - 그릴에서 구워져서 육즙이 가득하며, 특히 소스와 함께 곁들여 먹으면 더욱 맛있습니다.\n'
           '\n'
           '### 스테이크와 어울리는 와인 추천\n'
           '스테이크와 잘 어울리는 와인은 **레드 와인**입니다. 특히 **까베르네 소비뇽**이나 **말벡**이 추천됩니다. 이 '
           '와인들은 풍부한 탄닌과 과일의 깊은 맛이 스테이크의 육즙과 조화를 이루어, 더욱 풍부한 식사를 만들어줍니다.\n'
           '\n'
           '이 정보가 도움이 되었나요? 더 궁금한 점이 있다면 언제든지 물어보세요!'}


In [ ]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:
    try:
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))

        response = agent_executor.invoke({
            "input":message,
            "chat_history":chat_history[-2:]
        })

        return response['output']
    
    except Exception as e:
        print(f"Error occured: {str(e)}")
        return "죄송합니다. 응답을 생성하는 동안 오류가 발생했습니다. 다시 시도해 주세요."
    
examples_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

demo = gr.ChatInterface(
    fn = answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보, 추천, 음식 관련 질문에 답변해 드립니다",
    examples=examples_questions,
    theme=gr.themes.Soft()
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.')]시그니처 스테이크는 ₩35,000에 제공됩니다. 이 스테이크는 최상급 한우 등심을 사용하며, 21일간 건조 숙성하여 미디엄 레어로 조리됩니다. 육즙을 최대한 보존하면서 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 또한, 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

이 스테이크는 고급스러운 재료와 특별한 조리법으로 맛의 깊이를 더해주는 메뉴입니

In [ ]:
demo.close()